# Simulation of a single chain

In [1]:
import unittest
import feasst as fst
print(fst.version())

def calc_rg2(mc):
    conf = mc.system().configuration()
    rg2_sum = 0.0
    for i in range(conf.particle(0).num_sites() - 1):
        pos = conf.particle(0).site(i).position()
        for j in range(i+1, conf.particle(0).num_sites()):
            rg2_sum += pos.squared_distance(conf.particle(0).site(j).position())
    rg2 = rg2_sum / (conf.particle(0).num_sites()**2)
    return rg2

mc = fst.MonteCarlo()
mc.set(fst.MakeRandomMT19937(fst.args({"seed": "time"})))
mc.add(fst.MakeConfiguration(fst.args({
    "cubic_box_length": "40",
    "periodic0": "False",
    "periodic1": "False",
    "periodic2": "False",
    "particle_type": fst.install_dir() + "/plugin/chain/forcefield/chain5.fstprt"})))
mc.get_system().get_configuration().set_model_param("cutoff", 0, 20)
mc.add(fst.MakePotential(fst.MakeLennardJones(),
                         fst.MakeVisitModelIntra(fst.args({"cutoff": "1"}))))
mc.set(fst.MakeThermoParams(fst.args({"beta": "1", "chemical_potential0": "1"})))
mc.set(fst.MakeMetropolis())
mc.add(fst.MakeTrialAdd(fst.args({"particle_type": "0"})))
mc.run(fst.MakeRun(fst.args({"until_num_particles": "1"})))
mc.run(fst.MakeRemoveTrial(fst.args({"name": "TrialAdd"})))
mc.add(fst.MakeTrialPivot(fst.args({"weight": "1", "tunable_param": "20"})))
mc.add(fst.MakeTrialCrankshaft(fst.args({"weight": "1", "tunable_param": "20"})))
#mc.add(fst.MakeTrialReptate(fst.args({"weight": "1", "max_length": "1", "particle_type": "0"})))
mc.add(fst.MakeTrialGrowLinear(fst.MakeTrialComputeMove(),
                               fst.args({"weight": "1", "particle_type": "0", "num_steps": "3"})))
trials_per = str(int(1e4))
mc.add(fst.MakeLogAndMovie(fst.args({"trials_per": trials_per, "file_name": "chain"})))
mc.add(fst.MakeEnergy(fst.args({"trials_per_write": trials_per, "file_name": "chain_en.txt"})))
mc.add(fst.MakeCheckEnergy(fst.args({"trials_per": trials_per, "tolerance": str(1e-10)})))
mc.add(fst.MakeTune())
print('Potential energy of initial configuration:', mc.criteria().current_energy())
rg2 = fst.Accumulator()
mc.attempt(int(1e3)) # equilibrate
for n in range(int(1e5)):
    mc.attempt()
    rg2.accumulate(calc_rg2(mc))
            
class TestChain5RG(unittest.TestCase):
    def test_rg_en(self):
        print(rg2.str())
        self.assertAlmostEqual(0.8806, rg2.average(), delta=0.01)
        self.assertAlmostEqual(-2.050,
            fst.SeekAnalyze().reference("Energy", mc).accumulator().average(),
            delta=0.04)

v0.18.1-61-gca844632f9-hwh/txtin_factory
Potential energy of initial configuration: -0.19650552006989797


In [2]:
%time
unittest.main(argv=[''], verbosity=2, exit=False)

test_rg_en (__main__.TestChain5RG) ... 

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs
average,stdev,block_stdev,moment0,moment1,moment2,moment3,moment4,
0.87913777224692524,0.23281173227362806,0.0023682925306159953,100000,87913.777224692522,82708.398326250723,83471.583924272077,90291.042172433405,


ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!